# MovieLens database cleaning

## Extract needed dates

According to its ReadMe, this database has records from ratings dating from between January 09, 1995 and March 31, 2015. The Netflix database only ranges from October 1998 to December 2005, so that many entries are useless.

The numbers of distinct users goes from 138.493 to 52875.

In [1]:
import pandas as pd
import numpy as np

In [2]:
db = pd.read_csv("data/ML_dataset.csv")
print(db.shape)
a = (db["timestamp"] < 907200000) 
print("There are", sum(a), "reviews made before the 1st October 1998.")
b = (db["timestamp"] > 1135987200) 
print("There are", sum(b), "reviews made after the 31st December 2005.")

db.head()

(20000263, 4)
There are 2558956 reviews made before the 1st October 1998.
There are 8165131 reviews made after the 31st December 2005.


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [3]:
# Remove useless rows
a = (db["timestamp"] > 907200000) 
db = db[a]
print(db.shape)
b = (db["timestamp"] < 1135987200) 
db = db[b]
print(db.shape)


(17441307, 4)
(9276176, 4)


In [4]:
# Get number of distincts users
print("There are", len(db.userId.unique()), "users left.")

There are 52875 users left.


## Remove not matching movies



In [5]:
# Import movie titles

import csv

nf_movies = [];
with open('data/movie_titles_Netflix.csv', encoding="ISO-8859-1") as nf:
    for col1,col2,*col3 in csv.reader(nf):
        nf_movies.append(''.join(col3).lower().replace(" ",""))

ml_movies = [];
with open('data/movie_titles_ML.csv') as ml:
    for col1,col2,*col3 in csv.reader(ml):
        ml_movies.append((col1,''.join(col2).lower().replace(" ","")))
        
print(len(nf_movies))
print(len(ml_movies))

17770
27278


In [6]:
matches = {}
for movie in range(len(nf_movies)):
    #if any(nf_movies[movie] in q for s,q in ml_movies):
    for label, title in ml_movies:
        if nf_movies[movie] in title:
            matches[label] = movie + 1 
            break
        #matches.setdefault((int(s) for s,q in ml_movies if nf_movies[movie] in q), movie+1)

        
print("I found", len(matches), "matches.")

I found 6470 matches.


In [7]:
# Remove movies
idx2drop =[]
for ii,jj in enumerate(db['movieId']):
    if str(jj) not in matches:
        idx2drop.append(ii)
db.drop(db.index[idx2drop], inplace=True)
# for key in matches:
# db[db.movieId in matches.keys()]
    

In [8]:
print(db.shape)
print(len(idx2drop))
print(idx2drop[1:20])

(6139686, 4)
3136490
[9, 14, 17, 19, 22, 23, 27, 32, 35, 37, 40, 43, 44, 46, 47, 49, 53, 54, 55]
